# NLP: Embeddings
    - São únicas em termos de capturar o contexto das palavras e representá-las de forma que palavras com significados semelhantes sejam representadas com tipos semelhantes de Embeddings. Existem duas formas de calcular os embeddings.
    1. Skip Gram
    2. Continuous Bag of Words (CBOW)
    
    - Ambos os métodos fornecem valores de Embeddings que nada mais são do que pesos da camada oculta em uma rede neural.   
    1. O word2vec fornece os valores de incorporação para cada palavra
    2. enquanto doc2vec fornece os valores de incorporação para a frase inteira.

- Vamos usar um conjunto de dados de amostra para ilustrar como podemos criar Embeddings de sequência a partir de uma jornada de varejo online de usuários

In [55]:
pip install gensim 

Note: you may need to restart the kernel to use updated packages.


In [56]:
# Bibliotecas
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [57]:
url='C:/Users/Pc/Desktop/Hartb/machineLearning/PYSPARK/embedding_dataset.csv'

df = spark.read.csv(url,header=True, inferSchema=True)
df.show(5,False)

+----------------------------------------------------------------+------------+-------------------+------------+----------+---------+
|user_id                                                         |page        |timestamp          |visit_number|time_spent|converted|
+----------------------------------------------------------------+------------+-------------------+------------+----------+---------+
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|homepage    |2017-05-24 19:00:41|0           |0.16666667|1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info|2017-05-24 19:00:51|0           |0.4       |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info|2017-05-24 19:01:15|0           |0.31666666|1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info|2017-05-24 19:02:42|0           |0.6333333 |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0eb

In [58]:
df.count()

1096955

- O número total de registros no conjunto de dados é próximo a um milhão e há 0,1 milhão de usuários únicos. 
- O tempo gasto por cada usuário em cada uma das páginas da web também é rastreado junto com o status final se o usuário comprou o produto ou não

In [59]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- page: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- visit_number: integer (nullable = true)
 |-- time_spent: double (nullable = true)
 |-- converted: integer (nullable = true)



In [60]:
df.select('user_id').distinct().count()

104087

In [61]:
df.groupBy('page').count().orderBy('count',ascending=False).show(10,False)

+-------------+------+
|page         |count |
+-------------+------+
|product info |767131|
|homepage     |142456|
|added to cart|67087 |
|others       |39919 |
|offers       |32003 |
|buy          |24916 |
|reviews      |23443 |
+-------------+------+



In [62]:
df.select(['user_id','page','visit_number','time_spent','converted']).show(10,False)

+----------------------------------------------------------------+-------------+------------+-----------+---------+
|user_id                                                         |page         |visit_number|time_spent |converted|
+----------------------------------------------------------------+-------------+------------+-----------+---------+
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|homepage     |0           |0.16666667 |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info |0           |0.4        |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info |0           |0.31666666 |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info |0           |0.6333333  |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|product info |0           |0.15       |1        |
|8057ed24427be18922f640b20b60997e7d070946b6c8f48117ae4d6dad0ebb23|homepa

- A ideia das Embeddings de sequência é traduzir a série de etapas executadas pelo usuário durante sua jornada on-line em uma sequência de páginas, que pode ser usada para calcular pontuações de Embeddings.
- A primeira etapa é remover qualquer uma das páginas duplicadas consecutivas durante a jornada de um usuário
- Criamos uma coluna adicional que captura a página anterior de um usuário

- Window: é uma função no Spark que ajuda a aplicar certa lógica específica a um indivíduo ou grupo de linhas no conjunto de dados
- lag: retorna o valor da linha anterior de uma coluna em relação à linha atual 

In [63]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, lag, udf



In [64]:
w = Window.partitionBy("user_id").orderBy('timestamp')

df = df.withColumn("previous_page", lag("page", 1, 'started').over(w))

df.select('user_id','timestamp','previous_page','page').show(10,False)

+----------------------------------------------------------------+-------------------+-------------+------------+
|user_id                                                         |timestamp          |previous_page|page        |
+----------------------------------------------------------------+-------------------+-------------+------------+
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-05-02 07:01:04|started      |product info|
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-05-17 07:03:17|product info |product info|
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-05-28 11:02:22|product info |product info|
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-06-03 10:09:25|product info |product info|
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-06-10 16:42:04|product info |product info|
|0005ec769b11a4cf22c7124aecc4ab4bec496df4033e2987bfebfb77c512ea9c|2017-06-12 18:28:38|pr

In [65]:
def indicator(page, prev_page): 
    if page == prev_page: 
        return 0 
    else: return 1

In [66]:
from pyspark.sql.types import ArrayType,IntegerType
from pyspark.sql.functions import *

In [67]:
page_udf = udf(indicator,IntegerType())

df = df.withColumn("indicator",page_udf(col('page'),col('previous_page'))).withColumn('indicator_cummulative',sum(col('indicator')).over(w))

- O indicador cumulativo é a coluna para rastrear o número de páginas distintas durante a jornada do usuário.

In [68]:
df.select('previous_page','page','indicator','indicator_cummulative').show(20,False)

+-------------+------------+---------+---------------------+
|previous_page|page        |indicator|indicator_cummulative|
+-------------+------------+---------+---------------------+
|started      |product info|1        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product info|0        |1                    |
|product info |product i

- Continuamos criando novos objetos windows para particionar os dados ainda mais, a fim de construir as sequências para cada usuário.

In [69]:
w2 = Window.partitionBy(["user_id",'indicator_cummulative']).orderBy('timestamp')

df = df.withColumn('time_spent_cummulative', sum(col('time_spent')).over(w2))

df.select('timestamp','previous_page','page','indicator','indicator_cummulative','time_spent', 'time_spent_cummulative').show(20,False)

+-------------------+-------------+------------+---------+---------------------+-----------+----------------------+
|timestamp          |previous_page|page        |indicator|indicator_cummulative|time_spent |time_spent_cummulative|
+-------------------+-------------+------------+---------+---------------------+-----------+----------------------+
|2017-05-02 07:01:04|started      |product info|1        |1                    |0.08       |0.08                  |
|2017-05-17 07:03:17|product info |product info|0        |1                    |0.08       |0.16                  |
|2017-05-28 11:02:22|product info |product info|0        |1                    |0.08       |0.24                  |
|2017-06-03 10:09:25|product info |product info|0        |1                    |0.08       |0.32                  |
|2017-06-10 16:42:04|product info |product info|0        |1                    |0.08       |0.4                   |
|2017-06-12 18:28:38|product info |product info|0        |1             

- Na próxima etapa, calculamos o tempo agregado gasto em atividades semelhantes páginas para que apenas um único registro possa ser mantido para representar páginas consecutivas.

In [70]:
w3 = Window.partitionBy(["user_id",'indicator_cummulative']).orderBy(col('timestamp').desc())

df = df.withColumn('final_page',first('page').over(w3)).withColumn('final_time_spent',first('time_spent_cummulative').over(w3))

df.select(['time_spent_cummulative','indicator_cummulative','page','final_page','final_time_spent']).show(10,False)

+----------------------+---------------------+------------+------------+----------------+
|time_spent_cummulative|indicator_cummulative|page        |final_page  |final_time_spent|
+----------------------+---------------------+------------+------------+----------------+
|1.2                   |1                    |product info|product info|1.2             |
|1.1199999999999999    |1                    |product info|product info|1.2             |
|1.0399999999999998    |1                    |product info|product info|1.2             |
|0.9599999999999999    |1                    |product info|product info|1.2             |
|0.8799999999999999    |1                    |product info|product info|1.2             |
|0.7999999999999999    |1                    |product info|product info|1.2             |
|0.72                  |1                    |product info|product info|1.2             |
|0.64                  |1                    |product info|product info|1.2             |
|0.56     

In [71]:
aggregations=[]

aggregations.append(max(col('final_page')).alias('page_emb'))

aggregations.append(max(col('final_time_spent')).alias('time_spent_emb'))

aggregations.append(max(col('converted')).alias('converted_emb'))

df_embedding = df.select(['user_id','indicator_cummulative','final_page','final_time_spent','converted']).groupBy(['user_id','indicator_cummulative']).agg(*aggregations)

w4 = Window.partitionBy(["user_id"]).orderBy('indicator_cummulative')

w5 = Window.partitionBy(["user_id"]).orderBy(col('indicator_cummulative').desc())

- Por fim, usamos uma lista de coleta para combinar todas as páginas da jornada de um usuário em uma única lista e também para o tempo gasto. Como resultado, terminamos com a jornada do usuário na forma de uma lista de páginas e uma lista de tempo gasto.

In [72]:
df_embedding = df_embedding\
.withColumn('journey_page', collect_list(col('page_emb')).over(w4))\
.withColumn('journey_time_temp',collect_list(col('time_spent_emb')).over(w4)) \
.withColumn('journey_page_final',first('journey_page').over(w5))\
.withColumn('journey_time_final', first('journey_time_temp').over(w5)) \
.select(['user_id','journey_page_final','journey_time_final','converted_emb'])

- Continuamos com jornadas únicas de usuários. Cada usuário é representado por um único vetor de jornada e tempo gasto

In [73]:
df_embedding = df_embedding.dropDuplicates()

df_embedding.count()
 
df_embedding.select('user_id').distinct().count()
 
df_embedding.select('user_id','journey_page_final','journey_time_final').show(10)

+--------------------+--------------------+--------------------+
|             user_id|  journey_page_final|  journey_time_final|
+--------------------+--------------------+--------------------+
|0005ec769b11a4cf2...|      [product info]|               [1.2]|
|000b57de22d67187b...|      [product info]|[0.40333333600000...|
|001a13b2d3fae30b9...|[homepage, produc...|[0.26666668, 3.09...|
|0021689e622e8f268...|[homepage, produc...|[0.11666667, 8.26...|
|00323567146f62efb...|[homepage, produc...|[0.28333333, 0.26...|
|0038eeefa396cff29...|[product info, ho...|[0.58, 19.466667,...|
|003d29d24cff1d994...|[homepage, produc...|[0.25, 0.89666667...|
|00495ae8c90665343...|      [product info]|[1.4733333000000004]|
|004e96d0dc01f2541...|      [product info]|[3.5533333000000002]|
|0058982521702bf10...|      [product info]|       [2.210000036]|
+--------------------+--------------------+--------------------+
only showing top 10 rows



- Agora que temos as jornadas do usuário e a lista de tempo gasto, convertemos esse dataframe em um dataframe do Pandas e construímos um modelo word2vec usando essas sequências de jornada. Temos que instalar uma biblioteca gensim primeiro para usar o word2vec. Usamos o tamanho de incorporação de 100 para mantê-lo simples.

In [74]:
from gensim.models import Word2Vec

In [85]:
EMBEDDING_SIZE = 100

In [83]:
pd_df_emb0 = df_embedding.toPandas()

In [84]:
pd_df_embedding = pd_df_emb0.reset_index(drop=True)
pd_df_embedding

,user_id,journey_page_final,journey_time_final,converted_emb
0,0005ec769b11a4cf22c7124aecc4ab4bec496df4033e29...,[product info],[1.2],0
1,000b57de22d67187b62f4358a063ed49578cee26b49ec9...,[product info],[0.40333333600000004],0
2,001a13b2d3fae30b92d751c06f1edcfa222b1e488f96f7...,"[homepage, product info]","[0.26666668, 3.0999999860000003]",0
3,0021689e622e8f268c0da22075606a1b79aa228fab57f0...,"[homepage, product info]","[0.11666667, 8.266666559999999]",0
4,00323567146f62efb583248589bd17503177ea0fe16447...,"[homepage, product info, others, product info,...","[0.28333333, 0.266666665, 0.13333334, 0.650000...",0
...,...,...,...,...
104082,ffa8ffa4cab3553340955a52a9e84cea5eed111fa3d571...,"[homepage, product info]","[0.08, 8.91666694]",0
104083,ffb19abd0571ea668123df60b6c8838b0018eab717da7d...,"[homepage, product info, homepage, product info]","[0.16666667, 0.333333335, 0.6, 0.08]",0
104084,ffc57f3c1757a775cd0ed734f825368d67409d2cacca9f...,"[homepage, offers, product info, others, offer...","[0.36666667, 0.46666667, 0.51666665, 0.3166666...",0
104085,ffca62be7159e9d89cf8669b798e5b356943ed34f5944e...,"[homepage, buy, added to cart, buy, offers, pr...","[0.5, 0.31666666, 2.1666666, 0.2, 1.6333333, 0...",1


- Como podemos observar, o tamanho do vocabulário é 7 porque estávamos lidando com apenas 7 categorias de páginas. Cada uma dessas categorias de páginas agora pode ser representada com a ajuda de um vetor de incorporação de tamanho 100.

In [90]:
model = Word2Vec(pd_df_embedding['journey_page_final'], 100)
#CONTINUAR

TypeError: Both corpus_file and corpus_iterable must not be provided at the same time